# New Section

In [ ]:
!pip install simplejson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 KB 3.4 MB/s eta 0:00:00


In [ ]:
# Import the necessary libraries
import urllib.request
import shutil

# Set the URL of the file to download
url = "http://snap.stanford.edu/data/amazon/Electronics.txt.gz"

# Set the local file path to save the downloaded file
local_file_path = "/content/Electronics.txt.gz"

# Download the file from the URL and save it to the local file path
with urllib.request.urlopen(url) as response, open(local_file_path, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)


In [ ]:
import gzip
import simplejson
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix


In [ ]:
with gzip.open('/content/Electronics.txt.gz', 'rt') as file:
    lines = file.readlines()

In [ ]:
lines

In [ ]:
data = [{}]
for line in lines:
    if line.startswith('product/') or line.startswith('review/'):
        key, value = line.strip().split(':', 1)  # split on the first colon only
        data[-1][key] = value.strip()
    else:
        data.append({})

In [ ]:
df = pd.DataFrame(data)
 
 
# assuming your DataFrame is called "df"
filtered_df = df[df['review/helpfulness'] == '5/5']
filtered_df

In [ ]:

# re-index the filtered DataFrame and drop the old index
df=  filtered_df.reset_index(drop=True)
df

In [ ]:
df.isnull().sum()

In [ ]:
# drop rows with null values
df = df.dropna()

In [ ]:
df.isnull().sum()

In [ ]:
# randomly select 50,000 rows with a random state of 42
# sampled_df = df.sample(n=10000, random_state=13)
## sampled_df = df.tail(100001)

# process your sampled data frame here


In [ ]:
# Review=sampled_df.copy()
Review=df.copy()

In [ ]:
# # reset the index and delete the old index
# Review = Review.reset_index(drop=True)

In [ ]:
Review.head()

In [ ]:
# remove the specified columns
columns_to_remove = ['product/productId', 'product/title', 'product/price', 'review/userId',
                     'review/profileName', 'review/helpfulness', 'review/time', 'review/summary']
Review = Review.drop(columns=columns_to_remove)

In [ ]:
Review

In [ ]:
# replace decimal values in the review/score column with their corresponding integer values
Review['User_Mlabel'] = Review['review/score'].replace({'5.0': 4, '4.0': 3, '3.0': 2, '2.0': 1, '1.0': 0})
Review['User_label'] = Review['review/score'].replace({'5.0': 5, '4.0': 4, '3.0': 3, '2.0': 2, '1.0': 1})

In [ ]:
Review

In [ ]:
# Review.to_csv('Show.csv',index=False)

In [ ]:
# Review['review/text'][0]

In [ ]:
# Review['review/score'].dtype

In [ ]:
# Review['review/score'].unique()

In [ ]:
# Review['review/label'].dtype

In [ ]:
# Review['review/label'].unique()

In [ ]:
# # sum the unique values in the review/score column
# unique_scores = Review['review/label'].unique()
# total_score = sum(unique_scores)

In [ ]:
# total_score

In [ ]:
df=Review.copy()

In [ ]:
# rename the review/score and review/text columns
df = df.rename(columns={'review/score': 'User_Rating', 'review/text': 'Review'})


In [ ]:
df

In [ ]:
# plot a bar chart of the count of each score value
df['User_label'].value_counts().plot.bar()

# set the title and axis labels
plt.title('Count of Review label')
plt.xlabel('Score')
plt.ylabel('Count')

# display the plot
plt.show()

In [ ]:
#need equal length sentences
#plot hist of sentence length
plt.figure(figsize=(10, 5))
sns.histplot([len(s) for s in df.Review], bins=100)
plt.title('Sentence Length')
plt.show()

In [ ]:
Review_lengths = [len(s) for s in df['Review']]
df_Review_lengths = pd.DataFrame({'Review Length': Review_lengths})
df_Review_lengths

In [ ]:
# Generate random data
data = df_Review_lengths['Review Length']

# Fit a normal distribution to the data
mu, std = norm.fit(data)

# Create a histogram of the data
plt.hist(data, bins=20, density=True, alpha=0.5)

# Plot the probability density function (PDF) of the normal distribution
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)

# Add labels and title
plt.xlabel('Data')
plt.ylabel('Density')
plt.title('Histogram with Gaussian Fit')

# Show plot
plt.show()

In [ ]:
# Filter the DataFrame by the length of the 'Train Review' column being less than 4000
threshold_length = 4500
Threshold_df = df[df['Review'].apply(len) < threshold_length]

Threshold_df

##  VADER Sentiment Analysis 


In [ ]:
import nltk
nltk.download('vader_lexicon')


In [ ]:
# Import SentimentIntensityAnalyzer and create an sid object
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
# Obtain the sid scores for your review
sid.polarity_scores(Threshold_df['Review'][0])

In [ ]:
Threshold_df['Vader_Scores'] = Threshold_df['Review'].apply(lambda review: sid.polarity_scores(review))

Threshold_df['Vader_Compound']  = Threshold_df['Vader_Scores'].apply(lambda score_dict: score_dict['compound'])
Threshold_df

In [ ]:

def assign_category_Machine(vader_score):
    if vader_score < -0.6:
        return 0
    elif vader_score < -0.25:
        return 1
    elif vader_score < 0.25:
        return 2
    elif vader_score < 0.6:
        return 3
    else:
        return 4


In [ ]:
def assign_category(vader_score):
    if vader_score < -0.6:
        return 1
    elif vader_score < -0.25:
        return 2
    elif vader_score < 0.25:
        return 3
    elif vader_score < 0.6:
        return 4
    else:
        return 5

In [ ]:

def Vader_sentiment(vader_score):
    if vader_score < -0.1:
        return 0
    elif vader_score < 0.1:
        return 1
    else:
        return 2

In [ ]:

def Sentiment_Label(score):   # Based on customer Label
    if score < 2.5:  # 1 and 2 star > 0 class
        return 0
    elif score < 4.5: # 3 and 4 star > 1 class
        return 1
    else:              # 5 star > 2 class
        return 2


In [ ]:
Threshold_df['Vader_label'] = Threshold_df['Vader_Compound'].apply(assign_category)
Threshold_df['Vader_Mlabel'] = Threshold_df['Vader_Compound'].apply(assign_category_Machine)
Threshold_df['Sentiment_Label'] = Threshold_df['User_label'].apply(Sentiment_Label)
Threshold_df['Vader_sentiment'] = Threshold_df['Vader_Compound'].apply(Vader_sentiment)
Threshold_df

In [ ]:
# Count total occurrences of values in User_Mlabel column
user_mlabel_counts = Threshold_df['User_Mlabel'].value_counts()
print(user_mlabel_counts)

# Count total unique values in Sentiment_Label column
sentiment_label_counts = Threshold_df['Sentiment_Label'].value_counts()
print(sentiment_label_counts)

# Count total unique values in Sentiment_Label column
vader_label_counts = Threshold_df['Vader_sentiment'].value_counts()
print(vader_label_counts)

In [ ]:
print(Threshold_df['Vader_Mlabel'].dtypes)
print(Threshold_df['Vader_label'].dtypes)

In [ ]:
# remove the specified columns
columns_to_remove = ['Vader_Scores', 'User_Rating']
Threshold_df = Threshold_df.drop(columns=columns_to_remove)
Threshold_df

In [ ]:
# Define a function to preprocess the text data
def preprocess(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))   
    # Tokenize the text
    tokens = word_tokenize(text)  
    # Remove stopwords
    stopwords_list = stopwords.words('english')
    tokens = [token for token in tokens if token not in stopwords_list]  
    # Check for misspelled words
    # english_dict = enchant.Dict("en_US")
    # tokens = [token for token in tokens if english_dict.check(token)]
    #Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  
    # Join the tokens back into text
    text = ' '.join(tokens)  
    return text

In [ ]:
!pip install pyenchant


In [ ]:
!apt-get install libenchant-dev

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import enchant
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re



In [ ]:
# Apply the preprocess function to the text data
Threshold_df['Review'] = Threshold_df['Review'].apply(preprocess)
Threshold_df

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Split the data into training and testing sets with stratification
train_data, test_data = train_test_split(Threshold_df, test_size=0.15, random_state=16) #,14 stratify=Threshold_df['Sentiment_Label'])


In [ ]:
X_train =  train_data['Review']
y_train = train_data['Sentiment_Label']
X_test = test_data['Review']
y_test = test_data['Sentiment_Label']

In [ ]:
X_train[0]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object
count_vectorizer = CountVectorizer()

# Transform the data using CountVectorizer
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# Define the logistic regression model
lr_model = LogisticRegression()

# Train the model on the CountVectorizer transformed data
lr_model.fit(X_train_count, y_train)

# Use the trained model to make predictions on the test data
y_pred = lr_model.predict(X_test_count)

# Evaluate the performance of the model using classification report
print(classification_report(y_test, y_pred))


In [ ]:
# Create a TF-IDF vectorizer object
tfidf = TfidfVectorizer()
#tfidf = TfidfVectorizer(stop_words = 'english')
# Use the TF-IDF vectorizer to transform the data
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# # Define the logistic regression model with softmax activation
# lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=150)
lr_model = LogisticRegression()

# Train the model on the TF-IDF transformed data
lr_model.fit(X_train_tfidf, y_train)

# Use the trained model to make predictions on the test data
y_pred = lr_model.predict(X_test_tfidf)

# Evaluate the performance of the model using classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.83      0.64      0.73       476
           1       0.65      0.50      0.56       921
           2       0.73      0.88      0.80      1547

    accuracy                           0.72      2944
   macro avg       0.74      0.67      0.70      2944
weighted avg       0.72      0.72      0.71      2944



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid to search over
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['lbfgs', 'liblinear', 'saga'],
    'multi_class': ['ovr', 'multinomial']
}

# Create a logistic regression model with softmax activation
lr_model = LogisticRegression( max_iter=150)

# Create a GridSearchCV object to find the best hyperparameters
grid_search = GridSearchCV(lr_model, param_grid, cv=5, n_jobs=-1)

# Fit the GridSearchCV object on the training data
grid_search.fit(X_train_tfidf, y_train)

# Print the best hyperparameters found by GridSearchCV
print("Best hyperparameters: ", grid_search.best_params_)

# Use the best model to make predictions on the test data
y_pred = grid_search.predict(X_test_tfidf)

# Evaluate the performance of the best model using classification report
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
80 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solver l

Best hyperparameters:  {'C': 1, 'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'lbfgs'}
              precision    recall  f1-score   support

           0       0.83      0.64      0.72       476
           1       0.66      0.50      0.57       921
           2       0.73      0.88      0.80      1547

    accuracy                           0.72      2944
   macro avg       0.74      0.67      0.70      2944
weighted avg       0.72      0.72      0.71      2944



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define the hyperparameter distribution to search over
param_dist = {
    'max_depth': randint(1, 20),
    'min_samples_split': randint(2, 15),
    'min_samples_leaf': randint(1, 15),
    'criterion': ['gini', 'entropy']
}

# Create a decision tree model
dt_model = DecisionTreeClassifier()

# Create a RandomizedSearchCV object to find the best hyperparameters
random_search = RandomizedSearchCV(dt_model, param_distributions=param_dist, n_iter=20, cv=5, n_jobs=-1)

# Fit the RandomizedSearchCV object on the training data
random_search.fit(X_train_tfidf, y_train)

# Print the best hyperparameters found by RandomizedSearchCV
print("Best hyperparameters: ", random_search.best_params_)

# Use the best model to make predictions on the test data
y_pred = random_search.predict(X_test_tfidf)

# Evaluate the performance of the best model using classification report
print(classification_report(y_test, y_pred))


Best hyperparameters:  {'criterion': 'gini', 'max_depth': 14, 'min_samples_leaf': 3, 'min_samples_split': 7}
              precision    recall  f1-score   support

           0       0.61      0.34      0.44       476
           1       0.54      0.23      0.32       921
           2       0.60      0.89      0.72      1547

    accuracy                           0.59      2944
   macro avg       0.58      0.48      0.49      2944
weighted avg       0.58      0.59      0.55      2944



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define the hyperparameter distribution to search over
param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(1, 20),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 11),
    'criterion': ['gini', 'entropy']
}

# Create a random forest model
rf_model = RandomForestClassifier()

# Create a RandomizedSearchCV object to find the best hyperparameters
random_search = RandomizedSearchCV(rf_model, param_distributions=param_dist, n_iter=20, cv=5, n_jobs=-1)

# Fit the RandomizedSearchCV object on the training data
random_search.fit(X_train_tfidf, y_train)

# Print the best hyperparameters found by RandomizedSearchCV
print("Best hyperparameters: ", random_search.best_params_)

# Use the best model to make predictions on the test data
y_pred = random_search.predict(X_test_tfidf)

# Evaluate the performance of the best model using classification report
print(classification_report(y_test, y_pred))


Best hyperparameters:  {'criterion': 'entropy', 'max_depth': 16, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 875}
              precision    recall  f1-score   support

           0       1.00      0.01      0.02       476
           1       1.00      0.00      0.00       921
           2       0.53      1.00      0.69      1547

    accuracy                           0.53      2944
   macro avg       0.84      0.34      0.24      2944
weighted avg       0.75      0.53      0.37      2944



In [ ]:
# Define hyperparameter grids for each model
dtc_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
rfc_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


In [ ]:
gb_param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 1.0],
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [ ]:
xgb_param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'n_estimators': [50, 100, 200],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0, 0.1, 0.5, 1]
}


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier



In [ ]:
# Define the models to use
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gb = GradientBoostingClassifier()
xgb = XGBClassifier()

In [ ]:
# Perform grid search to find the best hyperparameters for each model
dtc_grid = GridSearchCV(dtc, dtc_param_grid, cv=5)
rfc_grid = GridSearchCV(rfc, rfc_param_grid, cv=5)
gb_grid = GridSearchCV(gb, gb_param_grid, cv=5)
xgb_grid = GridSearchCV(xgb, xgb_param_grid, cv=5)

NameError: ignored

In [ ]:
# Print the best hyperparameters found by GridSearchCV
print("Decision Tree: Best hyperparameters: ", dtc_grid.best_params_)

# Print the best hyperparameters found by GridSearchCV
print("Random Forest: Best hyperparameters: ", rfc_grid.best_params_)

# # Print the best hyperparameters found by GridSearchCV
# print("Gradient Boost: Best hyperparameters: ", gb_grid.best_params_)

# # Print the best hyperparameters found by GridSearchCV
# print("XG Boost: Best hyperparameters: ", xgb_grid.best_params_)


AttributeError: ignored

In [ ]:
dtc_grid.fit(X_train_tfidf, y_train)
dtc_pred = dtc_grid.predict(X_test_tfidf)
print('Decision Tree Classifier:\n', classification_report(y_test, dtc_pred)) 

KeyboardInterrupt: ignored

In [ ]:

rfc_grid.fit(X_train_tfidf, y_train)
rfc_pred = rfc_grid.predict(X_test_tfidf)
print('Random Forest Classifier:\n', classification_report(y_test, rfc_pred))

KeyboardInterrupt: ignored

In [ ]:
# Print the best hyperparameters found by GridSearchCV
print("Best hyperparameters: ", grid_search.best_params_)

In [ ]:

# gb_grid.fit(X_train_tfidf, y_train)
# gb_pred = gb_grid.predict(X_test_tfidf)
# print('Gradient Boosting Classifier:\n', classification_report(y_test, gb_pred))

In [ ]:

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

xgb_param_dist = {
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.6),
    'n_estimators': randint(100, 1000),
    'subsample': uniform(0.3, 0.7),
    'colsample_bytree': uniform(0.3, 0.7),
    'gamma': uniform(0, 1),
    'min_child_weight': randint(1, 10)
}

xgb_random = RandomizedSearchCV(xgb, param_distributions=xgb_param_dist, cv=5, n_iter=25)

xgb_random.fit(X_train_tfidf, y_train)
xgb_pred = xgb_random.predict(X_test_tfidf)
print('XGBoost Classifier:\n', classification_report(y_test, xgb_pred))


XGBoost Classifier:
               precision    recall  f1-score   support

           0       0.71      0.41      0.52       339
           1       0.42      0.05      0.09        96
           2       0.85      0.97      0.90      1553

    accuracy                           0.83      1988
   macro avg       0.66      0.48      0.51      1988
weighted avg       0.80      0.83      0.80      1988



In [ ]:
# # Print the best hyperparameters found by GridSearchCV
print("XG Boost: Best hyperparameters: ", xgb_random.best_params_)

XG Boost: Best hyperparameters:  {'colsample_bytree': 0.9684738517284268, 'gamma': 0.8670802814535783, 'learning_rate': 0.14017117521933503, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 216, 'subsample': 0.799249663265862}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats

# define the parameter distributions
gb_param_dists = {
    'learning_rate': stats.uniform(0.01, 0.3),
    'n_estimators': stats.randint(50, 500),
    'max_depth': stats.randint(2, 10),
    'min_samples_split': stats.uniform(0.01, 0.5),
    'min_samples_leaf': stats.uniform(0.01, 0.5),
    'subsample': stats.uniform(0.5, 0.5),
    'max_features': stats.uniform(0.5, 0.5)
}

# create an instance of the model
gb = GradientBoostingClassifier()

# create an instance of RandomizedSearchCV
gb_random = RandomizedSearchCV(gb, gb_param_dists, n_iter=25, cv=5)

# fit the randomized search to the data
gb_random.fit(X_train_tfidf, y_train)

gb_pred = gb_random.predict(X_test_tfidf)


print('XGBoost Classifier:\n', classification_report(y_test, gb_pred))

# # Print the best hyperparameters found by GridSearchCV
print("Gradient Boost: Best hyperparameters: ", gb_random.best_params_)

XGBoost Classifier:
               precision    recall  f1-score   support

           0       0.55      0.19      0.28       339
           1       0.00      0.00      0.00        96
           2       0.81      0.97      0.88      1553

    accuracy                           0.79      1988
   macro avg       0.45      0.39      0.39      1988
weighted avg       0.72      0.79      0.74      1988

Gradient Boost: Best hyperparameters:  {'learning_rate': 0.06343204557210377, 'max_depth': 2, 'max_features': 0.7277353514074585, 'min_samples_leaf': 0.0356098918083964, 'min_samples_split': 0.1693714411746995, 'n_estimators': 381, 'subsample': 0.5906382048817673}


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:


xgb_grid.fit(X_train_tfidf, y_train)
xgb_pred = xgb_grid.predict(X_test_tfidf)
print('XGBoost Classifier:\n', classification_report(y_test, xgb_pred))

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import classification_report

..........................................................

In [ ]:
# Set up the decision tree classifier
dtc = DecisionTreeClassifier()

# Set up the random forest classifier
rfc = RandomForestClassifier()

In [ ]:
# Set up the parameter grid for hyperparameter tuning
dtc_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40], # (0-100) 5 difference
    'min_samples_split': [2, 5, 10], # (2-20) 2 diffe
    'min_samples_leaf': [1, 2, 4] # (1-10) 1 diff
}

rfc_param_grid = {
    'n_estimators': [50, 100, 200], # (25-500) #25 diff
    'max_depth': [None, 10, 20, 30, 40], # (0-100) # 5 diff
    'min_samples_split': [2, 5, 10], # (2-20) 2 diffe
    'min_samples_leaf': [1, 2, 4] # (1-10) 1 diff
}

#xgboost, Gradient boost

In [ ]:
# XGBoost:

# xgb_param_grid = {
# 'learning_rate': [0.1, 0.01, 0.001], # (0-1) 0.1 diff
# 'max_depth': [3, 5, 7], # (1-10) 2 diff
# 'min_child_weight': [1, 3, 5], # (1-10) 2 diff
# 'gamma': [0, 0.1, 0.2], # (0-1) 0.1 diff
# 'subsample': [0.8, 1.0], # (0-1) 0.2 diff
# 'colsample_bytree': [0.8, 1.0], # (0-1) 0.2 diff
# 'n_estimators': [50, 100, 200], # (25-500) 25 diff
# 'reg_alpha': [0, 0.1, 0.5, 1], # (0-1) 0.25 diff
# 'reg_lambda': [0, 0.1, 0.5, 1] # (0-1) 0.25 diff
# }

# Gradient Boosting:

# gb_param_grid = {
# 'learning_rate': [0.1, 0.01, 0.001], # (0-1) 0.1 diff
# 'max_depth': [3, 5, 7], # (1-10) 2 diff
# 'min_samples_split': [2, 5, 10], # (2-20) 2 diff
# 'min_samples_leaf': [1, 2, 4], # (1-10) 1 diff
# 'subsample': [0.8, 1.0], # (0-1) 0.2 diff
# 'n_estimators': [50, 100, 200], # (25-500) 25 diff
# 'max_features': ['auto', 'sqrt', 'log2'], # [0.1-1] 0.3 diff
# 'alpha': [0.1, 0.3, 0.5, 0.7, 0.9], # (0-1) 0.2 diff
# 'loss': ['ls', 'lad', 'huber', 'quantile'] # Choice of loss function
# }

# Note that these are just example parameter grids, and the actual parameters you choose will depend on your specific use case and data. You may need to adjust these parameters based on your data size, model complexity, and other factors to achieve the best performance.







In [ ]:
# Set up the grid search object for decision tree classifier
dtc_grid = GridSearchCV(dtc, dtc_param_grid, cv=5, n_jobs=-1)

# Set up the grid search object for random forest classifier
rfc_grid = GridSearchCV(rfc, rfc_param_grid, cv=5, n_jobs=-1)

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [ ]:
# Wrap the code block for the random forest classifier with a with tf.device block
with tf.device('/GPU:0'):
# Fit the decision tree classifier to the data
  dtc_grid.fit(X_train_tfidf, y_train)



In [ ]:
# Wrap the code block for the random forest classifier with a with tf.device block
with tf.device('/GPU:0'):
 

# Fit the random forest classifier to the data
  rfc_grid.fit(X_train_tfidf, y_train)

In [ ]:
# Make predictions on the testing data using the decision tree classifier
dtc_pred = dtc_grid.predict(X_test_tfidf)

# Make predictions on the testing data using the random forest classifier
rfc_pred = rfc_grid.predict(X_test_tfidf)

In [ ]:
# Calculate the accuracy of the decision tree classifier
dtc_accuracy = accuracy_score(y_test, dtc_pred)

In [ ]:
# Calculate the accuracy of the random forest classifier
rfc_accuracy = accuracy_score(y_test, rfc_pred)

In [ ]:
# Print out the results
print("Decision Tree Classifier:")
print("Best Parameters: ", dtc_grid.best_params_)
print("Accuracy: ", dtc_accuracy)

print("Random Forest Classifier:")
print("Best Parameters: ", rfc_grid.best_params_)
print("Accuracy: ", rfc_accuracy)

..........................

# ..............................................................................................................................................................
